In [1]:
%matplotlib inline
%pylab inline
%load_ext autotime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import seaborn as sns
import os
import glob
import geopandas as gpd
import geopy
import plotly_express as px
import tqdm

from graphics import Point
from functools import reduce
from datetime import date
from datetime import datetime
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from tqdm.notebook import tqdm_notebook

Populating the interactive namespace from numpy and matplotlib
time: 1.86 s (started: 2021-12-16 20:31:11 -06:00)


In [2]:
from geopy.geocoders import Photon
geolocator = Photon(user_agent="measurements")

time: 16 ms (started: 2021-12-16 20:31:13 -06:00)


In [ ]:
feblistings = pd.read_csv('../data/feb2021listings.csv')
feblistings.shape

In [ ]:
feblistings.info()

In [ ]:
feblistings = feblistings.drop(['neighbourhood'], axis=1)

In [ ]:
feblistings.dropna(how='all', axis=1, inplace=True)

In [ ]:
feblistings.head()

In [ ]:
feblistings['latitude'].value_counts()

In [ ]:
feblistings['longitude'].value_counts()

In [ ]:
feblistings = pd.read_csv('../data/feb2021listings.csv')
feblistings.shape

In [ ]:
#march month not available in data

In [ ]:
Aprillistings = pd.read_csv('../data/April2021listings.csv')
Aprillistings.shape

In [ ]:
Aprillistings['longitude'].value_counts()

In [ ]:
Junelistings = pd.read_csv('../data/June2021listings.csv')
Junelistings.shape

In [ ]:
Junelistings['longitude'].value_counts()

In [ ]:
Julylistings = pd.read_csv('../data/July2021listings.csv')
Julylistings.shape

In [ ]:
Julylistings['longitude'].value_counts()

In [ ]:
Augustlistings = pd.read_csv('../data/August2021listings.csv')
Augustlistings.shape

In [ ]:
Augustlistings['longitude'].value_counts()

In [ ]:
Septemberlistings = pd.read_csv('../data/September2021listings.csv')
Septemberlistings.shape

In [ ]:
Septemberlistings['longitude'].value_counts()

In [ ]:
Octoberlistings = pd.read_csv('../data/October2021listings.csv')
Octoberlistings.shape

In [ ]:
Octoberlistings['longitude'].value_counts()

In [ ]:
Octoberlistings['latitude'].value_counts()

In [ ]:
#i cannot find the nan,nan values in any of the months - maybe it is duplicates from when the information was merged
# i will go ahead and drop all nan rows in my data and attempt to convert all addresses again
# if that does not work, i will need to convert addresses individually by month

In [3]:
all_filenames = [i for i in glob.glob('../data/monthlistings/*.{}'.format('csv'))]

time: 0 ns (started: 2021-12-16 20:31:25 -06:00)


In [4]:
print(all_filenames)

['../data/monthlistings\\April2021listings.csv', '../data/monthlistings\\August2021listings.csv', '../data/monthlistings\\December2020listings.csv', '../data/monthlistings\\feb2021listings.csv', '../data/monthlistings\\January2021listings.csv', '../data/monthlistings\\July2021listings.csv', '../data/monthlistings\\June2021listings.csv', '../data/monthlistings\\October2021listings.csv', '../data/monthlistings\\September2021listings.csv']
time: 0 ns (started: 2021-12-16 20:31:26 -06:00)


In [5]:
combined_months = pd.concat([pd.read_csv(f) for f in all_filenames ])

time: 266 ms (started: 2021-12-16 20:31:28 -06:00)


In [6]:
combined_months.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,6422,Nashville Charm,12172,Michele,NaN,District 6,36.17315,-86.73581,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,NaN
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,NaN,District 6,36.17826,-86.74162,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,NaN
2,39870,Close to Vanderbilt 2,171184,Evelyn,NaN,District 18,36.12523,-86.81278,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,NaN
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,NaN,District 12,36.16249,-86.58988,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,NaN
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,NaN,District 12,36.16102,-86.59207,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,NaN


time: 31 ms (started: 2021-12-16 20:31:29 -06:00)


In [7]:
#drops columns that have only NAN values
combined_months.dropna(how='all', axis=1, inplace=True)

time: 31 ms (started: 2021-12-16 20:31:35 -06:00)


In [8]:
combined_months = combined_months.drop(['neighbourhood'], axis=1)

time: 0 ns (started: 2021-12-16 20:31:37 -06:00)


In [9]:
combined_months['geom'] = combined_months['latitude'].map(str) + ',' + combined_months['longitude'].map(str)
combined_months['geom'][0]

0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
0    36.17315,-86.73581
Name: geom, dtype: object

time: 93 ms (started: 2021-12-16 20:31:41 -06:00)


In [10]:
combined_months.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55268 entries, 0 to 6239
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              55268 non-null  int64  
 1   name                            55268 non-null  object 
 2   host_id                         55268 non-null  int64  
 3   host_name                       55017 non-null  object 
 4   latitude                        55268 non-null  float64
 5   longitude                       55268 non-null  float64
 6   room_type                       55268 non-null  object 
 7   price                           55268 non-null  int64  
 8   minimum_nights                  55268 non-null  int64  
 9   number_of_reviews               55268 non-null  int64  
 10  last_review                     47985 non-null  object 
 11  reviews_per_month               47985 non-null  float64
 12  calculated_host_listings_count  5

In [11]:
combined_months.loc[:,"geom"]

0       36.17315,-86.73581
1       36.17826,-86.74162
2       36.12523,-86.81278
3       36.16249,-86.58988
4       36.16102,-86.59207
               ...        
6235    36.15691,-86.81922
6236    36.17879,-86.64087
6237    36.16347,-86.80429
6238    36.20127,-86.75325
6239    36.24438,-86.73639
Name: geom, Length: 55268, dtype: object

time: 0 ns (started: 2021-12-16 20:31:50 -06:00)


In [12]:
#drop duplicates 
combined_months.drop_duplicates(subset=['geom'], keep='first')

,id,name,host_id,host_name,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,geom
0,6422,Nashville Charm,12172,Michele,36.173150,-86.735810,Private room,40,30,674,2020-03-03,4.62,1,204,NaN,"36.17315,-86.73581"
1,25613,Room in Historic East Nashville Craftsman Home,95811,Nell,36.178260,-86.741620,Private room,75,2,510,2020-03-14,3.81,1,125,NaN,"36.17826,-86.74162"
2,39870,Close to Vanderbilt 2,171184,Evelyn,36.125230,-86.812780,Private room,60,1,262,2020-10-25,4.68,2,152,NaN,"36.12523,-86.81278"
3,258817,ButterflyRoom-private bath- open May 16,22296,Diana,36.162490,-86.589880,Private room,29,30,96,2019-12-14,0.84,7,343,NaN,"36.16249,-86.58988"
4,289242,"MorningstarHouse, monthly room- open may 1",22296,Diana,36.161020,-86.592070,Private room,28,30,76,2019-01-12,0.67,7,357,NaN,"36.16102,-86.59207"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6224,52316344,Deluxe Oasis of Comfort | 10min to DT | Pool +...,391537476,Kiesha,36.130227,-86.821282,Entire home/apt,175,3,0,NaN,NaN,1,73,0.0,"36.1302269680404,-86.82128215285468"
6225,52317418,A place of your own | 2BR in Nashville,368944610,Carolyn,36.077250,-86.723540,Entire home/apt,206,90,0,NaN,NaN,126,365,0.0,"36.07725,-86.72354"
6228,52317744,Stylish One Bedroom in an Amenity packed build...,391945218,Matt,36.158980,-86.774940,Entire home/apt,274,30,0,NaN,NaN,13,51,0.0,"36.15898,-86.77494"
6233,52329163,Landing | Modern Apartment with Amazing Amenit...,263502162,Landing,36.163470,-86.595230,Entire home/apt,122,30,0,NaN,NaN,5,338,0.0,"36.16347,-86.59523"


time: 62 ms (started: 2021-12-16 20:33:57 -06:00)


In [ ]:
api_key = 

In [ ]:
geolocator = GoogleV3(api_key=google_key)

In [ ]:
locator = Nominatim(user_agent='myGeocoder', timeout=10)
rgeocode = RateLimiter(locator.reverse, min_delay_seconds=0.1)

In [ ]:
from tqdm._tqdm_notebook import tqdm_notebook

In [ ]:
tqdm_notebook.pandas()

In [ ]:
combined_months['address'] = combined_months['geom'].progress_apply(rgeocode)
combined_months.head()